In [1]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_30268\65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [2]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[4608513006, 7913763000, 5174224005, 233526500..."
1,Name,object,14,0.00,779583,"[BLD Enterprises, Inc., CREATIVE MONOGRAMMING ..."
2,City,object,30,0.00,32581,"[PHILADELPHIA, ST CHARLES, SALT LAKE CITY, PLY..."
3,State,object,14,0.00,51,"[WI, SC, MO, CO]"
4,Zip,int64,0,0.00,33611,"[50501, 12207, 43065, 13036, 70816]"
5,Bank,object,1559,0.17,5802,"[CITIBANK, N.A., BANK OF AMERICA NATL ASSOC, R..."
6,BankState,object,1566,0.17,56,"[CA, ID, IL, IA]"
7,NAICS,int64,0,0.00,1312,"[812111, 337125, 448210, 0, 444190]"
8,ApprovalDate,object,0,0.00,9859,"[14-Mar-03, 17-Dec-07, 28-Feb-08, 25-Jan-02, 7..."
9,ApprovalFY,object,0,0.00,70,"[1992, 2008, 2003, 2005]"


In [3]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [4]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [5]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)].astype(object)
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [6]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [7]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [8]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [9]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [10]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [11]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [12]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [13]:
df.shape

(675026, 13)

In [14]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[72, 44-45, 62]"
1,Term,object,0,0.0,377,"[84, 300]"
2,NoEmp,object,0,0.0,499,"[2, 1, 40]"
3,NewExist,object,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,object,0,0.0,216,"[0, 5, 2]"
5,RetainedJob,object,0,0.0,323,"[5, 0, 7, 6]"
6,UrbanRural,object,0,0.0,3,"[1, 2]"
7,RevLineCr,object,0,0.0,2,[N]
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int32,0,0.0,16880,"[150000, 35000, 390000, 270300]"


In [15]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_0.csv


In [16]:
df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_30268\3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [17]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [18]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [19]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [20]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [21]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [22]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [23]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [24]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [25]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [26]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [27]:
df.shape

(460816, 13)

In [28]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[23, 71, 54, 44-45]"
1,Term,int64,0,0.0,373,"[19, 240, 81]"
2,NoEmp,int64,0,0.0,432,"[4, 2, 5]"
3,NewExist,object,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,192,"[0, 1]"
5,RetainedJob,int64,0,0.0,276,"[6, 0, 3]"
6,UrbanRural,int64,0,0.0,3,"[1, 0, 2]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int32,0,0.0,12824,"[256300, 10000, 66000, 50000]"


In [29]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_1.csv


In [30]:
df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_30268\3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [31]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [32]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [33]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [34]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [35]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [36]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [37]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [38]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [39]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [40]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [41]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [42]:
df.shape

(675026, 13)

In [43]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[72, 42, 62, 81]"
1,Term,int64,0,0.0,377,"[202, 87, 240, 95, 84]"
2,NoEmp,int64,0,0.0,499,"[1, 5, 3, 30]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 2]"
5,RetainedJob,int64,0,0.0,323,"[0, 1]"
6,UrbanRural,int64,0,0.0,3,"[0, 1]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int32,0,0.0,16880,"[75000, 30000, 213000, 10000, 400000]"


In [44]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_2.csv


In [45]:
df = pd.read_csv(imported_csv_path)

C:\Users\33679\AppData\Local\Temp\ipykernel_30268\3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [46]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[3835285007, 3370914009, 1558615001, 577451300..."
1,Name,object,14,0.00,779583,"[KYOTO JAPANESE RESTAURANT, C&R FASHION TRENDS..."
2,City,object,30,0.00,32581,"[EL CENTRO, ANAHEIM, SALINAS, HYANNIS, WICHITA]"
3,State,object,14,0.00,51,"[CA, NY, IN, TX]"
4,Zip,int64,0,0.00,33611,"[60619, 55118, 59405, 17241, 85029]"
5,Bank,object,1559,0.17,5802,"[JPMORGAN CHASE BANK NATL ASSOC, BANK OF AMERI..."
6,BankState,object,1566,0.17,56,"[NC, OR, OH, SD, KY]"
7,NAICS,int64,0,0.00,1312,"[541330, 522310, 0, 235310, 621210]"
8,ApprovalDate,object,0,0.00,9859,"[20-Jun-06, 30-Dec-98, 6-Jun-05, 27-Oct-86, 26..."
9,ApprovalFY,object,0,0.00,70,"[1995, 2001, 2000, 1990, 2010]"


In [47]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [48]:
df['NewExist'] = df['NewExist'].isin({1.0, 2.0}).astype(int).astype(str).astype(object)

In [49]:
is_valide = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide).astype(object)

In [50]:
is_valide = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide).astype(object)

In [51]:
df = df.dropna(subset='NAICS')
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [52]:
mapping = {0: 'unknown', 1: 'urban', 2: 'rural'}
df['UrbanRural'] = df['UrbanRural'].map(mapping).astype(object)

In [53]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1).astype(object)
df = df.drop('FranchiseCode', axis=1)

In [54]:
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y').dt.strftime('%d-%m-%y').astype(object)

In [55]:
df['ApprovalDate'].unique()

array(['28-02-97', '07-02-06', '25-03-97', ..., '25-02-97', '26-02-97',
       '27-02-97'], dtype=object)

In [56]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:]).astype(object)
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [57]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [58]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [59]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [60]:
df['MIS_Status'].value_counts()

MIS_Status
P I F     550334
CHGOFF    145335
Name: count, dtype: int64

In [61]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0).astype(bool)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [62]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [63]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,Name,object,0,0.0,611131,"[67TH AVENUE MARKET, BLAZE ENGINEERING INC, MI..."
1,City,object,0,0.0,28860,"[BEACON, SAINT LOUIS, BRONX, INDIANAPOLIS, CON..."
2,State,object,0,0.0,51,"[NJ, NY, IL, CA]"
3,Bank,object,0,0.0,5221,"[BANK OF AMERICA CALIFORNIA N.A, BBCN BANK, CA..."
4,BankState,object,0,0.0,55,"[CA, OK, NC, GA]"
5,NAICS,object,0,0.0,20,"[81, 23, 53, 52]"
6,ApprovalDate,object,0,0.0,7367,"[14-06-11, 15-05-02, 22-05-03, 20-09-05, 01-02..."
7,ApprovalFY,object,0,0.0,29,"[2004, 2006, 2005, 2007, 2010]"
8,Term,int64,0,0.0,378,"[59, 61, 120, 84, 48]"
9,NoEmp,int64,0,0.0,502,"[7, 8, 5, 2, 1]"


In [64]:
df.shape

(696572, 20)

In [65]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset\SBA_Cleaned_Pure.csv
